In [18]:
import os
import numpy as np
import cvxpy as cp
import cv2 as cv
from sklearn.preprocessing import normalize, StandardScaler
from sklearn.decomposition import PCA

In [19]:
def random_face(feat_mat: np.ndarray, dim: int) -> np.ndarray:
    kernel = np.random.normal(size=[feat_mat.shape[1], dim])
    kernel = normalize(kernel)
    return feat_mat @ kernel

In [20]:
# def pca_feature(X_train: np.ndarray, X_test, dim: int)
#     pca = PCA(n_components=100, svd_solver="randomized", whiten=True)
#     pca.fit(X_train)
#     return pca.transform(X_train), pca.transform(X_test)

In [21]:
def cone_programming(feat_mat: np.ndarray, input: np.ndarray, epsilon: int) -> np.ndarray:
    lin_para = cp.Variable(feat_mat.shape[0])
    cons = cp.SOC(epsilon, lin_para @ feat_mat - input)
    obj_func = cp.Minimize(cp.norm(lin_para, p=1))
    prob = cp.Problem(obj_func, [cons])
    prob.solve()
    return lin_para.value

In [22]:
def sc_index(k: int, num_per_id: int, x: np.ndarray):
    l1_norm = np.sum(np.abs(x))
    ratio = 0
    for i in range(k):
        this_ratio = np.sum(np.abs(x[i*num_per_id:(i+1)*num_per_id]))
        this_ratio /= l1_norm
        ratio = np.max([ratio, this_ratio])
    sc_idx = (k*ratio - 1) / (k-1)
    return sc_idx

In [23]:
nomask_path = "D:\\Code_uc\\face_reco\\output\\dataset_nomask"
mask_path = "D:\\Code_uc\\face_reco\\output\\dataset_mask"
ag_nomask_path = "D:\\Code_uc\\face_reco\\output\\augment_nomask"
ag_mask_path = "D:\\Code_uc\\face_reco\\output\\augment_mask"

In [24]:
X = []
for i, subject_name in enumerate(os.listdir(ag_nomask_path)):
    subject_images_dir = os.path.join(ag_nomask_path, subject_name)
    for img_name in os.listdir(subject_images_dir):
        img_path = os.path.join(subject_images_dir, img_name)
        img = cv.imread(img_path, cv.IMREAD_GRAYSCALE)
        img = np.reshape(img, [-1])
        X.append(img)

X = np.asarray(X)

In [25]:
# foo = np.arange(9)
# sc_index(3, 3, foo)

In [26]:
# idx = 5
# input = feat_mat[idx]
# X = np.vstack([feat_mat[:idx], feat_mat[idx+1:]])

In [27]:
# feat_mat = random_face(X, feat_size)
# feat_mat = normalize(feat_mat)

In [28]:
# scaler = StandardScaler()
# feat_mat = scaler.fit_transform(X)
# pca = PCA(n_components=50, svd_solver="randomized", whiten=True)
# pca.fit(feat_mat)
# feat_mat = pca.transform(feat_mat)# 

In [29]:
X_rj = X[2400:]
X_train = np.ndarray(shape=[0, 22500])
X_ac = np.ndarray(shape=[0, 22500])
for i in range(40):
    X_train = np.vstack([X_train, X[i*60:i*60+44]])
    X_ac = np.vstack([X_ac, X[i*60+44:(i+1)*60]])

In [30]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_ac = scaler.transform(X_ac)
X_rj = scaler.transform(X_rj)
pca = PCA(n_components=50, svd_solver="randomized", whiten=True)
pca.fit(X_train)
X_train = pca.transform(X_train)
X_ac = pca.transform(X_ac)
X_rj = pca.transform(X_rj)

In [31]:
sc_rj = []
sc_ac = []
for rj_input in X_rj:
    para = cone_programming(X_train, rj_input, 5e-4)
    sc_idx = sc_index(40, 44, para)
    sc_rj.append(sc_idx)

for ac_input in X_ac:
    para = cone_programming(X_train, ac_input, 5e-4)
    sc_idx = sc_index(40, 44, para)
    sc_ac.append(sc_idx)

sc_rj = np.asarray(sc_rj)
sc_ac = np.asarray(sc_ac)


In [36]:
rj_num = np.count_nonzero(sc_rj < 0.16)
ac_num = np.count_nonzero(sc_ac > 0.16)
print(rj_num, ac_num)

515 486


In [33]:
# idx = 5
# input = feat_mat[5]
# X_train = np.vstack([feat_mat[:idx], feat_mat[idx+1:]])
# para = cone_programming(X_train, input, 1e-4)
# big_item = np.nonzero(np.abs(para) > 0.05)

In [34]:
# para[big_item[0]]